#### Task 1: Get info for one Movie and store in a python Dictionary

#### Import Libraries

In [1]:
from bs4 import BeautifulSoup as bs
import requests

#### Load Webpage

In [2]:
r = requests.get("https://en.wikipedia.org/wiki/Toy_Story_3")
soup = bs(r.content)
content = soup.prettify()
#print(content)

In [6]:
info_box = soup.find(class_="infobox vevent")
#print(info_box.prettify())
info_rows = info_box.find_all("tr")

#for row in info_rows:
    #print(row.prettify())



In [13]:
def get_content_value(row_data):
    if row_data.find("li"):
        return [li.get_text(" ", strip=True).replace('\xa0', ' ') for li in row_data.find_all("li")]
    else:
        return row_data.get_text(" ", strip=True).replace('\xa0', ' ') 

    
movie_info = {}

for index, row in enumerate(info_rows):
    if index == 0:
        movie_info['title'] = row.find("th").get_text(" ", strip=True) 
    elif index == 1:
        continue
    else:
        content_key = row.find("th").get_text(" ", strip=True)
        content_value = get_content_value(row.find("td"))
        movie_info[content_key] = content_value
        
movie_info


{'title': 'Toy Story 3',
 'Directed by': 'Lee Unkrich',
 'Screenplay by': 'Michael Arndt',
 'Story by': ['John Lasseter', 'Andrew Stanton', 'Lee Unkrich'],
 'Produced by': 'Darla K. Anderson',
 'Starring': ['Tom Hanks',
  'Tim Allen',
  'Joan Cusack',
  'Don Rickles',
  'Wallace Shawn',
  'John Ratzenberger',
  'Estelle Harris',
  'Ned Beatty',
  'Michael Keaton',
  'Jodi Benson',
  'John Morris'],
 'Cinematography': ['Jeremy Lasky', 'Kim White'],
 'Edited by': 'Ken Schretzmann',
 'Music by': 'Randy Newman',
 'Production companies': ['Walt Disney Pictures', 'Pixar Animation Studios'],
 'Distributed by': 'Walt Disney Studios Motion Pictures',
 'Release dates': ['June 12, 2010 ( 2010-06-12 ) ( Taormina Film Fest )',
  'June 18, 2010 ( 2010-06-18 ) (United States)'],
 'Running time': '103 minutes [1]',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$200 million [1]',
 'Box office': '$1.067 billion [1]'}

#### Task 2: Scape all the other movie information box from the Disney wikipedia page

In [14]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")
soup_2 = bs(r.content)
content = soup_2.prettify()
#print(content)

#### Tast 2-a: Get links for the different movies

In [17]:
movies = soup_2.select(".wikitable.sortable i")
movies[0:6]

[<i><a href="/wiki/Snow_White_and_the_Seven_Dwarfs_(1937_film)" title="Snow White and the Seven Dwarfs (1937 film)">Snow White and the Seven Dwarfs</a></i>,
 <i><a href="/wiki/Pinocchio_(1940_film)" title="Pinocchio (1940 film)">Pinocchio</a></i>,
 <i><a href="/wiki/Fantasia_(1940_film)" title="Fantasia (1940 film)">Fantasia</a></i>,
 <i><a href="/wiki/The_Reluctant_Dragon_(1941_film)" title="The Reluctant Dragon (1941 film)">The Reluctant Dragon</a></i>,
 <i><a href="/wiki/Dumbo" title="Dumbo">Dumbo</a></i>,
 <i><a href="/wiki/Bambi" title="Bambi">Bambi</a></i>]

In [47]:
def get_content_value(row_data):
    if row_data.find("li"):
        return [li.get_text(" ", strip=True).replace('\xa0', ' ') for li in row_data.find_all("li")]
    
    elif row_data.find("br"):
        return [text for text in row_data.stripped_strings]
    
    else:
        return row_data.get_text(" ", strip=True).replace('\xa0', ' ')    
    
def clean_tags(soup):
    for tag in soup.find_all(["sup", "span"]):
        tag.decompose()


def get_info_box(url):
        
    r = requests.get(url)
    soup = bs(r.content)
    
    clean_tags(soup)
    
    info_box = soup.find(class_="infobox vevent")
    info_rows = info_box.find_all("tr")
    
    
    

    
    movie_info = {}
    for index, row in enumerate(info_rows):
        if index == 0:
            movie_info['title'] = row.find("th").get_text(" ", strip=True) 
        
        else:
            header = row.find('th')
            if header:
                content_key = row.find("th").get_text(" ", strip=True)
                content_value = get_content_value(row.find("td"))
                movie_info[content_key] = content_value    
            
    return movie_info

In [45]:
#Test Cleaning solution
get_info_box('https://en.wikipedia.org/wiki/Toy_Story_3')

{'title': 'Toy Story 3',
 'Directed by': 'Lee Unkrich',
 'Screenplay by': 'Michael Arndt',
 'Story by': ['John Lasseter', 'Andrew Stanton', 'Lee Unkrich'],
 'Produced by': 'Darla K. Anderson',
 'Starring': ['Tom Hanks',
  'Tim Allen',
  'Joan Cusack',
  'Don Rickles',
  'Wallace Shawn',
  'John Ratzenberger',
  'Estelle Harris',
  'Ned Beatty',
  'Michael Keaton',
  'Jodi Benson',
  'John Morris'],
 'Cinematography': ['Jeremy Lasky', 'Kim White'],
 'Edited by': 'Ken Schretzmann',
 'Music by': 'Randy Newman',
 'Production companies': ['Walt Disney Pictures', 'Pixar Animation Studios'],
 'Distributed by': ['Walt Disney Studios', 'Motion Pictures'],
 'Release dates': ['June 12, 2010 ( Taormina Film Fest )',
  'June 18, 2010 (United States)'],
 'Running time': '103 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$200 million',
 'Box office': '$1.067 billion'}

In [48]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")
soup_2 = bs(r.content)
movies = soup_2.select(".wikitable.sortable i a")

base_path = 'https://en.wikipedia.org/'

movie_info_list = []
for index, movie in enumerate(movies):
    #if index == 10:
        #break
    try:
        relative_path = movie['href']
        full_path = base_path + relative_path
        title = movie['title']
        
        movie_info_list.append(get_info_box(full_path))
        
    except Exception as e:
        print(movie.get_text())
        print(e)

Zorro the Avenger
'NoneType' object has no attribute 'find'
The Sign of Zorro
'NoneType' object has no attribute 'find'
Spirited Away
'NoneType' object has no attribute 'find'
Howl's Moving Castle
'NoneType' object has no attribute 'find'
Ponyo
'NoneType' object has no attribute 'find'
Tales from Earthsea
'NoneType' object has no attribute 'find'
The Secret World of Arrietty
'NoneType' object has no attribute 'find'
The Beatles: Get Back – The Rooftop Concert
'NoneType' object has no attribute 'find'
Wish
'NoneType' object has no attribute 'find_all'
Elio
'NoneType' object has no attribute 'find_all'
61
'NoneType' object has no attribute 'find_all'
All Night Long
'NoneType' object has no attribute 'find'
Big Thunder Mountain Railroad
'NoneType' object has no attribute 'find_all'
Keeper of the Lost Cities
'NoneType' object has no attribute 'find_all'
Muppet Man
'NoneType' object has no attribute 'find_all'
One Thousand and One Nights
'NoneType' object has no attribute 'find_all'
Shrunk


In [49]:
len(movie_info_list)

525

#### Save Data as Json

In [75]:
import json

def save_data(title, data):
    with open(title, 'w', encoding = "utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

In [76]:
import json

def load_data(title):
    with open(title, encoding = "utf-8") as f:
        return json.load(f)
    

In [50]:
save_data("Disney_Movies_semi_cleaned.json", movie_info_list)

### Data Cleaning!

- ~Clean up References e.g [1];~
- ~Convert runtime into integers;~
- ~Convert dates into datetime;~
- ~Split long strings;~
- ~Convert Budget and Box Office to numbers.~

In [52]:
movie_info_list = load_data("Disney_Movies_semi_cleaned.json")

In [7]:
#[movie.get('Running time', 'N/A') for movie in movie_info_list]

In [57]:
def minute_to_int(running_time):
    if running_time == 'N/A':
        return None
    
    if isinstance(running_time, list):
        return int(running_time[0].split(' ')[0])
    else:
        return int(running_time.split(' ')[0])
    

for movie in movie_info_list:
    movie["Running time(int)"] = minute_to_int(movie.get('Running time', 'N/A'))
        

In [59]:
movie_info_list[-10]

{'title': 'National Treasure: Book of Secrets',
 'Directed by': 'Jon Turteltaub',
 'Screenplay by': ['Cormac Wibberley', 'Marianne Wibberley'],
 'Story by': ['Gregory Poirier',
  'Cormac Wibberley',
  'Marianne Wibberley',
  'Ted Elliott',
  'Terry Rossio'],
 'Based on': ['Characters', 'by', 'Jim Kouf', 'Oren Aviv', 'Charles Segars'],
 'Produced by': ['Jerry Bruckheimer', 'Jon Turteltaub'],
 'Starring': ['Nicolas Cage',
  'Jon Voight',
  'Harvey Keitel',
  'Ed Harris',
  'Diane Kruger',
  'Justin Bartha',
  'Bruce Greenwood',
  'Helen Mirren'],
 'Cinematography': ['John Schwartzman', 'Amir Mokri'],
 'Edited by': ['William Goldenberg', 'David Rennie'],
 'Music by': 'Trevor Rabin',
 'Production companies': ['Walt Disney Pictures',
  'Jerry Bruckheimer Films',
  'Junction Entertainment',
  'Saturn Films'],
 'Distributed by': 'Walt Disney Studios Motion Pictures',
 'Release date': ['December 21, 2007'],
 'Running time': '124 minutes',
 'Country': 'United States',
 'Language': 'English',
 '

In [8]:
#[movie.get('Budget','N/A') for movie in movie_info_list]

In [70]:
import re

amounts = r"thousand|million|billion"
number = r"\d+(,\d{3})*\.*\d*"

value_re = rf"\${number}"
word_re = rf"\${number}(-|\sto\s)?({number})?\s({amounts})"
'''
money_conversion("$12.2 million") --> 12200000
money_conversion("$790,000") --> 790000
'''


def word_to_value(word):
    value_dict = {'thousand':1000, 'million': 1000000, 'billion': 1000000000}
    return value_dict[word]



def parse_word_syntax(string):
    value_string = re.search(number, string).group()
    value = float(value_string.replace(',', ''))
    word = re.search(amounts, string, flags=re.I).group().lower()
    word_value = word_to_value(word)
    return value*word_value


def parse_value_syntax(string):
    value_string = re.search(number, string).group()
    value = float(value_string.replace(',', ''))
    return value


def money_conversion(money):
    
    if money == "N/A":
        return None

    if isinstance(money, list):
        money = money[0]

    word_syntax = re.search(word_re, money, flags=re.I)
    value_syntax = re.search(value_re, money)

    if word_syntax:
        return parse_word_syntax(word_syntax.group())
    
    elif value_syntax:
        return parse_value_syntax(value_syntax.group())
    
    else:
        return None

In [69]:
for movie in movie_info_list:
    movie["Budget(float)"] = money_conversion(movie.get('Budget', 'N/A'))
    movie["Box office(float)"] = money_conversion(movie.get('Box office', 'N/A'))

#### Convert dates to datetime

In [71]:
movie_info_list[-10]

{'title': 'National Treasure: Book of Secrets',
 'Directed by': 'Jon Turteltaub',
 'Screenplay by': ['Cormac Wibberley', 'Marianne Wibberley'],
 'Story by': ['Gregory Poirier',
  'Cormac Wibberley',
  'Marianne Wibberley',
  'Ted Elliott',
  'Terry Rossio'],
 'Based on': ['Characters', 'by', 'Jim Kouf', 'Oren Aviv', 'Charles Segars'],
 'Produced by': ['Jerry Bruckheimer', 'Jon Turteltaub'],
 'Starring': ['Nicolas Cage',
  'Jon Voight',
  'Harvey Keitel',
  'Ed Harris',
  'Diane Kruger',
  'Justin Bartha',
  'Bruce Greenwood',
  'Helen Mirren'],
 'Cinematography': ['John Schwartzman', 'Amir Mokri'],
 'Edited by': ['William Goldenberg', 'David Rennie'],
 'Music by': 'Trevor Rabin',
 'Production companies': ['Walt Disney Pictures',
  'Jerry Bruckheimer Films',
  'Junction Entertainment',
  'Saturn Films'],
 'Distributed by': 'Walt Disney Studios Motion Pictures',
 'Release date': ['December 21, 2007'],
 'Running time': '124 minutes',
 'Country': 'United States',
 'Language': 'English',
 '

In [9]:
#[movie.get('Release date','N/A') for movie in movie_info_list]

In [85]:
from datetime import datetime


dates = [movie.get('Release date','N/A') for movie in movie_info_list]

def clean_date(date):
    return date.split('(')[0].strip()

def date_conversion(date):
    if isinstance(date, list):
        date = date[0]
    
    if date =='N/A':
        return None 
    
    date_str = clean_date(date)
    fmts = ["%B %d, %Y", "%d %B %Y"]  
    for fmt in fmts:
        try:
             return  datetime.strptime(date_str, fmt)
        except:
            pass
    return None
    
    

In [86]:
for movie in movie_info_list:
    movie["Release Date(Datetime)"] = date_conversion(movie.get('Release date', 'N/A'))

In [87]:
movie_info_list[-10]

{'title': 'National Treasure: Book of Secrets',
 'Directed by': 'Jon Turteltaub',
 'Screenplay by': ['Cormac Wibberley', 'Marianne Wibberley'],
 'Story by': ['Gregory Poirier',
  'Cormac Wibberley',
  'Marianne Wibberley',
  'Ted Elliott',
  'Terry Rossio'],
 'Based on': ['Characters', 'by', 'Jim Kouf', 'Oren Aviv', 'Charles Segars'],
 'Produced by': ['Jerry Bruckheimer', 'Jon Turteltaub'],
 'Starring': ['Nicolas Cage',
  'Jon Voight',
  'Harvey Keitel',
  'Ed Harris',
  'Diane Kruger',
  'Justin Bartha',
  'Bruce Greenwood',
  'Helen Mirren'],
 'Cinematography': ['John Schwartzman', 'Amir Mokri'],
 'Edited by': ['William Goldenberg', 'David Rennie'],
 'Music by': 'Trevor Rabin',
 'Production companies': ['Walt Disney Pictures',
  'Jerry Bruckheimer Films',
  'Junction Entertainment',
  'Saturn Films'],
 'Distributed by': 'Walt Disney Studios Motion Pictures',
 'Release date': ['December 21, 2007'],
 'Running time': '124 minutes',
 'Country': 'United States',
 'Language': 'English',
 '

In [88]:
save_data('Disney_Movies_Cleaned(complete).json', movie_info_list)

TypeError: Object of type datetime is not JSON serializable

In [89]:
import pickle

def save_with_pickle(name, data):

    with open(name, 'wb') as f:
        pickle.dump(data, f)

        
def load_with_pickle(name):
    with open(name, 'rb') as f:
        return pickle.load(f)
    


In [90]:
save_with_pickle("Disney_Movies_Data_Cleaned.pickle", movie_info_list)

In [91]:
data = load_with_pickle("Disney_Movies_Data_Cleaned.pickle")

In [92]:
data[5]

{'title': 'Bambi',
 'Directed by': ['Supervising director',
  'David Hand',
  'Sequence directors',
  'James Algar',
  'Samuel Armstrong',
  'Graham Heid',
  'Bill Roberts',
  'Paul Satterfield',
  'Norman Wright'],
 'Story by': ['Story direction',
  'Perce Pearce',
  'Story adaptation',
  'Larry Morey',
  'Story development',
  'Vernon Stallings',
  'Melvin Shaw',
  'Carl Fallberg',
  'Chuck Couch',
  'Ralph Wright'],
 'Based on': ['Bambi, a Life in the Woods', 'by', 'Felix Salten'],
 'Produced by': 'Walt Disney',
 'Starring': 'see below',
 'Music by': ['Frank Churchill', 'Edward H. Plumb'],
 'Production company': 'Walt Disney Productions',
 'Distributed by': 'RKO Radio Pictures',
 'Release dates': ['August 9, 1942 ( London )',
  'August 21, 1942 (United States)'],
 'Running time': '70 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$858,000',
 'Box office': '$267.4 million',
 'Running time(int)': 70,
 'Budget(float)': 858000.0,
 'Box office(float)': 2673999